In [1]:
import boto3

dynamodb = boto3.resource(
    'dynamodb',
    region_name='us-west-2',
    endpoint_url='http://localhost:8000', # importante para conexión local
    aws_access_key_id='fakeMyKeyId',
    aws_secret_access_key='fakeSecretAccessKey'
)


In [2]:
# Ejemplo: listar tablas
print(list(dynamodb.tables.all()))

[dynamodb.Table(name='Data-Engineering-ProductCatalog')]


In [3]:
import boto3
client = boto3.client(
        "dynamodb",
        endpoint_url="http://localhost:8000",  # 🔸 DynamoDB Local
        region_name="us-west-2",               # 🔹 Requerido por boto3 (puede ser cualquiera)
        aws_access_key_id="fakeMyKeyId",       # 🔹 Cualquier valor
        aws_secret_access_key="fakeSecretKey"  # 🔹 Cualquier valor
    )

In [5]:
client.list_tables()

EndpointConnectionError: Could not connect to the endpoint URL: "http://localhost:8000/"

In [ ]:

def create_table_db(table_name: str, **kwargs):

    response = client.create_table(TableName=table_name, **kwargs)

    waiter = client.get_waiter("table_exists")
    waiter.wait(TableName=table_name)

    return response


In [ ]:
def create_table_db(table_name: str, **kwargs):
    """
    Crea una tabla en DynamoDB usando un cliente de boto3 previamente configurado.

    Parámetros:
    -----------
    table_name : str
        El nombre de la tabla que se quiere crear.
    
    **kwargs :
        Otros parámetros necesarios para definir la tabla, como:
            - AttributeDefinitions
            - KeySchema
            - ProvisionedThroughput
            - GlobalSecondaryIndexes (opcional)
            - LocalSecondaryIndexes (opcional)

    Retorna:
    --------
    dict
        La respuesta del cliente de boto3 con la descripción de la tabla creada.
    
    Nota:
    -----
    Esta función espera que el objeto `client` ya esté creado en el ámbito global.
    Además, utiliza un "waiter" de boto3 para esperar a que la tabla esté disponible antes de continuar.
    """

    # Llamamos a la función create_table del cliente para crear la tabla con los parámetros recibidos.
    response = client.create_table(TableName=table_name, **kwargs)

    # Obtenemos un "esperador" (waiter) que se bloquea hasta que la tabla exista (es decir, esté lista).
    waiter = client.get_waiter("table_exists")
    waiter.wait(TableName=table_name)

    # Devolvemos la respuesta que contiene los detalles de la tabla recién creada.
    return response


In [4]:
COURSE_PREFIX = "Data-Engineering"

In [5]:
capacity_units = {'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}

product_catalog_table = {'table_name': f'{COURSE_PREFIX}-ProductCatalog',
                         'kwargs': {
                             'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}],
                             'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}],
                             'ProvisionedThroughput': capacity_units
                         }
                        }

forum_table = {'table_name': f'{COURSE_PREFIX}-Forum',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'Name', 'KeyType': 'HASH'}],
                    'AttributeDefinitions': [{'AttributeName': 'Name', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
              }

thread_table = {'table_name': f'{COURSE_PREFIX}-Thread',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'ForumName', 'KeyType': 'HASH'}, 
                                  {'AttributeName': 'Subject', 'KeyType': 'RANGE'}],
                    'AttributeDefinitions': [{'AttributeName': 'ForumName', 'AttributeType': 'S'},
                                             {'AttributeName': 'Subject', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
               }

reply_table = {'table_name': f'{COURSE_PREFIX}-Reply',
                'kwargs': {
                    'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}, 
                                  {'AttributeName': 'ReplyDateTime', 'KeyType': 'RANGE'}],
                    'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'S'},
                                             {'AttributeName': 'ReplyDateTime', 'AttributeType': 'S'}],
                    'ProvisionedThroughput': capacity_units
                }
              }

In [6]:
response = create_table_db(table_name=product_catalog_table['table_name'], **product_catalog_table["kwargs"]) 
print(response)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Id', 'AttributeType': 'N'}], 'TableName': 'Data-Engineering-ProductCatalog', 'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}], 'TableStatus': 'ACTIVE', 'CreationDateTime': datetime.datetime(2025, 5, 28, 18, 25, 10, 749000, tzinfo=tzlocal()), 'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()), 'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()), 'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 10, 'WriteCapacityUnits': 5}, 'TableSizeBytes': 0, 'ItemCount': 0, 'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Data-Engineering-ProductCatalog', 'DeletionProtectionEnabled': False}, 'ResponseMetadata': {'RequestId': '62630d32-7616-401d-b8df-0d2e5132d9f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Jetty(12.0.14)', 'date': 'Wed, 28 May 2025 21:25:07 GMT', 'x-amzn-requestid': '62630d32-7616-401d-b8df-0d2e5132d9f1', 'cont

In [16]:
import json

In [17]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "561",
            "content-type": "application/x-amz-json-1.0",
            "date": "Wed, 28 May 2025 21:25:07 GMT",
            "server": "Jetty(12.0.14)",
            "x-amz-crc32": "2211520945",
            "x-amzn-requestid": "62630d32-7616-401d-b8df-0d2e5132d9f1"
        },
        "HTTPStatusCode": 200,
        "RequestId": "62630d32-7616-401d-b8df-0d2e5132d9f1",
        "RetryAttempts": 0
    },
    "TableDescription": {
        "AttributeDefinitions": [
            {
                "AttributeName": "Id",
                "AttributeType": "N"
            }
        ],
        "CreationDateTime": "2025-05-28 18:25:10.749000-03:00",
        "DeletionProtectionEnabled": false,
        "ItemCount": 0,
        "KeySchema": [
            {
                "AttributeName": "Id",
                "KeyType": "HASH"
            }
        ],
        "ProvisionedThroughput": {
            "LastDecreaseDateTime

In [18]:
# dir(client)

In [9]:
client.list_tables()

{'TableNames': ['Data-Engineering-ProductCatalog'],
 'ResponseMetadata': {'RequestId': 'e8b3e427-fe43-471e-b7a1-e53b976d4bbb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Jetty(12.0.14)',
   'date': 'Wed, 28 May 2025 21:27:31 GMT',
   'x-amzn-requestid': 'e8b3e427-fe43-471e-b7a1-e53b976d4bbb',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '1891198489',
   'content-length': '50'},
  'RetryAttempts': 0}}

In [19]:
# dir(dynamodb)